## Importing libraries

In [6]:
import random

import numpy as np
import pandas as pd
import torch
from scipy.special import softmax
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    mean_absolute_error,
)
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping, LRScheduler
from skorch.dataset import ValidSplit
from torch import cuda, nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
from torchvision.transforms import Compose, ToTensor

from dlordinal.datasets import FGNet
from dlordinal.losses import (
    BetaLoss,
    BinomialLoss,
    EMDLoss,
    GeometricLoss,
    TriangularLoss,
    WKLoss,
)
from dlordinal.metrics import accuracy_off1, amae, mmae, ranked_probability_score

## Dataset
Download `FGNet` dataset.

In [7]:
fgnet_train = FGNet(
    root="./datasets",
    download=True,
    train=True,
    transform=Compose([ToTensor()]),
)

fgnet_test = FGNet(
    root="./datasets",
    download=True,
    train=False,
    transform=Compose([ToTensor()]),
)

num_classes = len(fgnet_train.classes)
classes = fgnet_train.classes
targets = fgnet_train.targets

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cuda device


## Metrics 
Metrics to evaluate different ordinal losses.

In [8]:
def calculate_metrics(y_true, y_pred):

    if np.allclose(np.sum(y_pred, axis=1), 1):
        y_pred_proba = y_pred
    else:
        y_pred_proba = softmax(y_pred, axis=1)

    y_pred_max = np.argmax(y_pred, axis=1)

    # Metrics
    amae_metric = amae(y_true, y_pred)
    mmae_metric = mmae(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred_max)
    acc = accuracy_score(y_true, y_pred_max)
    acc_1off = accuracy_off1(y_true, y_pred)
    qwk = cohen_kappa_score(y_true, y_pred_max, weights="quadratic")
    rps = ranked_probability_score(y_true, y_pred_proba)

    metrics = {
        "ACC": acc,
        "1OFF": acc_1off,
        "MAE": mae,
        "QWK": qwk,
        "AMAE": amae_metric,
        "MMAE": mmae_metric,
        "RPS": rps,
    }

    for key, value in metrics.items():
        print(f"{key}: {value}")

    print(confusion_matrix(y_true, y_pred_max))

    return metrics

## Experiment
We want to do a brief comparison of several (ordinal) losses using `PyTorch` and `Skorch`
with ResNet18, a pre-trained convolutional neural network, as the model architecture. 
Concretely, we compare Cross Entropy (CE) Loss with several ordinal approaches from the dlordinal library:

- Cross Entropy (CE) Loss
- Squared Earth Mover's Distance (EMD) Loss [1]
- Weighted Kappa Loss [2]
- Binomial Cross Entropy Loss  [3]
- Triangular Cross Entropy Loss [4]
- Beta Cross Entropy Loss [5]
- Geometric Cross Entropy Loss [6]

[1] Hou, L., Yu, C. P., & Samaras, D. (2016). Squared earth mover's distance-based loss for training deep neural networks. arXiv preprint arXiv:1611.05916. 

[2] de La Torre, J., Puig, D., & Valls, A. (2018). Weighted kappa loss function for multi-class classification of ordinal data in deep learning. Pattern Recognition Letters, 105, 144-154.

[3] Liu, X., Fan, F., Kong, L., Diao, Z., Xie, W., Lu, J., & You, J. (2020). Unimodal regularized neuron stick-breaking for ordinal classification. Neurocomputing, 388, 34-44.

[4] Vargas, V. M., Gutiérrez, P. A., Barbero-Gómez, J., & Hervás-Martínez, C. (2023). Soft labelling based on triangular distributions for ordinal classification. Information Fusion, 93, 258-267.

[5] Vargas, V. M., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). Unimodal regularisation based on beta distribution for deep ordinal regression. Pattern Recognition, 122, 108310.

[6] Haas, S., & Hüllermeier, E. (2023, September). Rectifying bias in ordinal observational data using unimodal label smoothing. In Joint European Conference on Machine Learning and Knowledge Discovery in Databases (pp. 3-18). Cham: Springer Nature Switzerland.



In [9]:
# Loss functions
losses = [
    CrossEntropyLoss().to(device),
    EMDLoss(num_classes=num_classes).to(device),
    WKLoss(num_classes=num_classes, use_logits=True).to(device),
    BinomialLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(device),
    TriangularLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(device),
    BetaLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(device),
    GeometricLoss(
        base_loss=nn.CrossEntropyLoss(),
        num_classes=num_classes,
        alphas=[0.15, 0.35, 0.35, 0.35, 0.35, 0.15],
    ).to(device),
]

# Evaluate each loss K times with different seeds to obtain a more robust result
K = 5

result = pd.DataFrame()


def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # Add deterministic settings
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


for loss_fn in losses:

    for k in range(K):
        # Make results reproducible
        seed = k
        set_seed(seed)  # Use K different seeds

        # Initialize ResNet18 model
        model = models.resnet18(weights="IMAGENET1K_V1")
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        model.to(device)

        # Skorch estimator
        estimator = NeuralNetClassifier(
            module=model,
            criterion=loss_fn,
            optimizer=Adam,
            lr=0.001,
            max_epochs=30,
            # verbose=0,
            train_split=ValidSplit(
                0.1, random_state=seed
            ),  # Use 10% of the data for validation
            callbacks=[
                EarlyStopping(patience=5, monitor="valid_loss"),
                LRScheduler(policy=ReduceLROnPlateau, patience=3, factor=0.5),
            ],
            device=device,
            batch_size=200,
        )

        print("#" + str(k) + " " + type(loss_fn).__name__)

        estimator.fit(
            X=fgnet_train, y=torch.tensor(fgnet_train.targets, dtype=torch.long)
        )

        test_probs = estimator.predict_proba(fgnet_test)

        metrics = calculate_metrics(np.array(fgnet_test.targets), test_probs)
        print("\n")

        df = pd.DataFrame([metrics])
        df["iteration"] = k
        df["loss"] = type(loss_fn).__name__

        result = pd.concat([result, df], ignore_index=True)

#0 CrossEntropyLoss
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.7114       0.3457        1.7738  1.3052
      2        0.7154       0.4321        1.7623  1.1908
      3        0.1940       0.4691        1.4878  1.1691
      4        0.0581       0.5556        1.6346  1.1309
      5        0.0123       0.5926        1.5629  1.1622
      6        0.0103       0.5679        1.7050  1.1971
      7        0.0040       0.6173        1.5860  1.2049
Stopping since valid_loss has not improved in the last 5 epochs.
ACC: 0.5522388059701493
1OFF: 0.8805970149253731
MAE: 0.6169154228855721
QWK: 0.7271115915976316
AMAE: 0.7727272727272728
MMAE: 1.7857142857142858
RPS: 0.51440736818413
[[15  7  0  0  0  0]
 [ 5 52  2  0  1  0]
 [ 0 20  6  7  0  0]
 [ 0  8  9 20  5  0]
 [ 0  2  5  8 14  1]
 [ 0  2  3  3  2  4]]


#1 CrossEntropyLoss
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  ----

## Result
Displays the mean results for each loss over the K iterations.

In [10]:
result.set_index(["loss", "iteration"], inplace=True)
result.groupby("loss").agg("mean").style.highlight_max(
    axis=0, subset=["ACC", "1OFF", "QWK"], color="green"
).highlight_min(axis=0, subset=["MAE", "AMAE", "MMAE", "RPS"], color="green")

,ACC,1OFF,MAE,QWK,AMAE,MMAE,RPS
loss,,,,,,,
BetaLoss,0.558209,0.921393,0.539303,0.793731,0.647857,1.528571,0.394261
BinomialLoss,0.519403,0.924378,0.562189,0.796562,0.656515,1.428571,0.443245
CrossEntropyLoss,0.504478,0.847761,0.706468,0.679063,0.835007,1.757143,0.566009
EMDLoss,0.534328,0.886567,0.624876,0.724371,0.745541,1.685714,0.501293
GeometricLoss,0.568159,0.912438,0.541294,0.786425,0.637525,1.400000,0.399524
TriangularLoss,0.578109,0.907463,0.532338,0.791809,0.645368,1.442857,0.404042
WKLoss,0.460697,0.882587,0.685572,0.732517,0.782179,1.377143,0.617337
